## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from string import punctuation
import json

## Data Preprocessing

In [2]:
# Load data
with open("./sentiment.txt") as f:
    reviews = f.read()
    
reviews

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0\nNot sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0\nAttempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0\nVery little music or anything to speak of.  \t0\nThe best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1\nThe rest of the movie lacks art, charm, meaning... If it\'s about emptiness, it works I guess because it\'s empty.  \t0\nWasted two hours.  \t0\nSaw the movie today and thought it was a good effort, good messages for kids.  \t1\nA bit predictable.  \t0\nLoved the casting of Jimmy Buffet as the science teacher.  \t1\nAnd those baby owls were adorable.  \t1\nThe movie showed a lot of Florida at it\'s best, made it look very appealing.  \t1\nThe S

In [3]:
data = pd.DataFrame([review.split('\t') for review in reviews.split('\n')], columns=['Review', 'Sentiment'])
data.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [4]:
# Shuffle the data
data = data.sample(frac=1)
data.head()

,Review,Sentiment
1707,Would not recommend to others.,0
2123,Bad Quality.,0
927,It was a long time that i didn't see a so char...,1
411,Her role was played well.,1
2068,Great for iPODs too.,1


In [5]:
def split_words_reviews(data):
    text = list(data['Review'].values)
    clean_text = []
    for t in text:
        clean_text.append(t.translate(str.maketrans('', '', punctuation)).lower().rstrip())
    tokenized = [word_tokenize(x) for x in clean_text]
    all_text = []
    for tokens in tokenized:
        for t in tokens:
            all_text.append(t)
    return tokenized, set(all_text)

In [6]:
# Tokenize
reviews, vocab = split_words_reviews(data)

print(reviews[0])
print(vocab)

['would', 'not', 'recommend', 'to', 'others']
{'where', 'caballeros', 'francis', 'bread', 'hitchcock', 'irritating', 'focused', 'tired', 'possible', 'due', 'blandest', 'miniseries', 'fifties', 'tell', 'casting', 'off', 'drunk', 'adapters', 'awsome', 'occupied', 'female', 'rude', 'hello', 'sliced', 'jobs', 'stereo', 'same', 'pcs', '2006', 'blown', 'calamari', 'ringtones', 'life', 'foodservice', 'dropped', 'teaches', 'rated', 'overly', 'wifi', 'trimmed', 'falsely', 'allowing', 'options', 'curve', 'something', 'pleasant', 'handset', 'spices', 'coziness', 'weekly', 'fits', 'panna', 'upper', 'screenjames', 'earbugs', 'iam', 'suck', 'instance', 'moods', 'cheekbones', 'wellit', 'playing', 'callings', 'floor', 'anyways', 'our', 'sim', 'inconsiderate', 'grade', 'childlike', 'mystifying', 'low', 'levels', 'buddy', '400', 'nuts', 'akin', 'speakerphone', 'hut', 'spotty', 'grease', 'cars', 'gooodd', 'pockets', 'caring', 'fest', 'mistake', 'whoa', 'lox', 'conditions', 'bathrooms', 'control', 'recept

In [8]:
def create_dictionaries(words):
    word_to_int_dict = {w:i+1 for i, w in enumerate(words)}
    word_to_int_dict[''] = 0  # Ensuring the empty string is included and mapped to 0
    int_to_word_dict = {i:w for w, i in word_to_int_dict.items()}
    return word_to_int_dict, int_to_word_dict

In [10]:
# Creating vocabulary
word_to_int_dict, int_to_word_dict = create_dictionaries(vocab)

print(int_to_word_dict)
print(word_to_int_dict)

{1: 'where', 2: 'caballeros', 3: 'francis', 4: 'bread', 5: 'hitchcock', 6: 'irritating', 7: 'focused', 8: 'tired', 9: 'possible', 10: 'due', 11: 'blandest', 12: 'miniseries', 13: 'fifties', 14: 'tell', 15: 'casting', 16: 'off', 17: 'drunk', 18: 'adapters', 19: 'awsome', 20: 'occupied', 21: 'female', 22: 'rude', 23: 'hello', 24: 'sliced', 25: 'jobs', 26: 'stereo', 27: 'same', 28: 'pcs', 29: '2006', 30: 'blown', 31: 'calamari', 32: 'ringtones', 33: 'life', 34: 'foodservice', 35: 'dropped', 36: 'teaches', 37: 'rated', 38: 'overly', 39: 'wifi', 40: 'trimmed', 41: 'falsely', 42: 'allowing', 43: 'options', 44: 'curve', 45: 'something', 46: 'pleasant', 47: 'handset', 48: 'spices', 49: 'coziness', 50: 'weekly', 51: 'fits', 52: 'panna', 53: 'upper', 54: 'screenjames', 55: 'earbugs', 56: 'iam', 57: 'suck', 58: 'instance', 59: 'moods', 60: 'cheekbones', 61: 'wellit', 62: 'playing', 63: 'callings', 64: 'floor', 65: 'anyways', 66: 'our', 67: 'sim', 68: 'inconsiderate', 69: 'grade', 70: 'childlike',

In [11]:
with open('word_to_int_dict.json', 'w') as fp:
    json.dump(word_to_int_dict, fp)

In [12]:
print(np.max([len(x) for x in reviews]))
print(np.mean([len(x) for x in reviews]))

70
11.783333333333333


In [13]:
def pad_text(tokenized_reviews, seq_length):
    reviews = []
    for review in tokenized_reviews:
        if len(review) >= seq_length:
            reviews.append(review[:seq_length])
        else:
            reviews.append(['']*(seq_length-len(review)) + review)
    return np.array(reviews)

In [14]:
# Pad and encode reviews
seq_length = 50
padded_sentences = pad_text(reviews, seq_length)
encoded_sentences = np.array([[word_to_int_dict[word] for word in review] for review in padded_sentences])
encoded_sentences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 3077, 3687, 2373, 1296, 1618])

In [15]:
# Prepare labels
labels = np.array([int(x) for x in data['Sentiment'].values])

## Model Building

In [18]:
class SentimentLSTM(Model):
    def __init__(self, n_vocab, n_embed, n_hidden, n_output, n_layers, drop_p=0.8):
        super(SentimentLSTM, self).__init__()
        self.embedding = layers.Embedding(input_dim=n_vocab, output_dim=n_embed)
        self.lstm = layers.LSTM(n_hidden, return_sequences=True, return_state=True, dropout=drop_p)
        self.dropout = layers.Dropout(drop_p)
        self.fc = layers.Dense(n_output, activation='sigmoid')

    def call(self, x):
        x = self.embedding(x)
        lstm_out, _, _ = self.lstm(x)
        x = self.dropout(lstm_out)
        x = self.fc(x)
        return x[:, -1]

n_vocab = len(word_to_int_dict)
n_embed = 50
n_hidden = 100
n_output = 1
n_layers = 2

# Instantiate the model
model = SentimentLSTM(n_vocab, n_embed, n_hidden, n_output, n_layers)

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

## Model Training

In [19]:
train_ratio = 0.8
valid_ratio = (1 - train_ratio) / 2

total = len(encoded_sentences)
train_cutoff = int(total * train_ratio)
valid_cutoff = int(total * (1 - valid_ratio))

train_x, train_y = encoded_sentences[:train_cutoff], labels[:train_cutoff]
valid_x, valid_y = encoded_sentences[train_cutoff:valid_cutoff], labels[train_cutoff:valid_cutoff]
test_x, test_y = encoded_sentences[valid_cutoff:], labels[valid_cutoff:]

batch_size = 1

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(batch_size)
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(batch_size)

In [20]:
model.fit(train_dataset, validation_data=valid_dataset, epochs=5)

Epoch 1/5
2400/2400 [==============================] - 115s 45ms/step - loss: 0.6770 - accuracy: 0.5696 - val_loss: 0.6116 - val_accuracy: 0.6767
Epoch 2/5
2400/2400 [==============================] - 94s 39ms/step - loss: 0.5411 - accuracy: 0.7221 - val_loss: 0.5467 - val_accuracy: 0.7100
Epoch 3/5
2400/2400 [==============================] - 107s 45ms/step - loss: 0.4093 - accuracy: 0.8292 - val_loss: 0.4556 - val_accuracy: 0.7967
Epoch 4/5
2400/2400 [==============================] - 111s 46ms/step - loss: 0.3380 - accuracy: 0.8733 - val_loss: 0.4155 - val_accuracy: 0.8233
Epoch 5/5
2400/2400 [==============================] - 97s 40ms/step - loss: 0.2843 - accuracy: 0.8938 - val_loss: 0.4107 - val_accuracy: 0.8333


In [21]:
model.summary()

Model: "sentiment_lstm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  270150    
                                                                 
 lstm_1 (LSTM)               multiple                  60400     
                                                                 
 dropout_1 (Dropout)         multiple                  0         
                                                                 
 dense_1 (Dense)             multiple                  101       
                                                                 
Total params: 330651 (1.26 MB)
Trainable params: 330651 (1.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Evaluation and Prediction

In [22]:
def preprocess_review(review, word_to_int_dict, seq_length=50):
    review = review.translate(str.maketrans('', '', punctuation)).lower().rstrip()
    tokenized = word_tokenize(review)
    if len(tokenized) >= seq_length:
        review = tokenized[:seq_length]
    else:
        review = ['']*(seq_length-len(tokenized)) + tokenized

    final = []
    for token in review:
        try:
            final.append(word_to_int_dict[token])
        except:
            final.append(word_to_int_dict[''])
    return np.array(final)

In [23]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}")

300/300 [==============================] - 3s 9ms/step - loss: 0.3507 - accuracy: 0.8600
Test Loss: 0.3507
Test Accuracy: 0.86


In [24]:
def predict_review(review):
    review_encoded = preprocess_review(review, word_to_int_dict)
    review_encoded = np.expand_dims(review_encoded, axis=0)
    prediction = model.predict(review_encoded)
    msg = "This is a positive review." if prediction >= 0.5 else "This is a negative review."
    print(msg)
    print(f'Prediction = {prediction[0][0]}')

In [25]:
predict_review("The film was good")
predict_review("It was not good")

1/1 [==============================] - 1s 1s/step
This is a positive review.
Prediction = 0.9905424118041992
1/1 [==============================] - 0s 61ms/step
This is a negative review.
Prediction = 0.09490545839071274


In [26]:
predict_review("the film was not that good")

1/1 [==============================] - 0s 48ms/step
This is a negative review.
Prediction = 0.18270081281661987
